In [19]:
import os
import random
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Utils

In [20]:
def show(image):
    plt.figure()
    if len(image.shape) == 2:
        plt.imshow(image, cmap='gray')
    else:
        img2 = image[:,:,::-1]
        plt.imshow(img2)

In [21]:
# Fonction pour redimensionner une image pour une taille cible
def resize_image(image, target_size):
    # Récupérer la hauteur et la largeur de l'image d'origine
    height, width = image.shape[:2]

    # Redimensionner l'image en conservant le rapport d'aspect
    if height > width:
        new_height = target_size
        new_width = int(width * (target_size / height))
    else:
        new_width = target_size
        new_height = int(height * (target_size / width))

    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

    return resized_image

In [22]:
import math

def overlap_percentage(circle1, circle2):
    # Coordonnées et rayons des cercles
    x1, y1 = circle1[0]
    x2, y2 = circle2[0]
    r1 = circle1[1]
    r2 = circle2[1]

    # Calcul de la distance entre les centres des cercles
    distance_centers = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    # Si les cercles ne se chevauchent pas du tout
    if distance_centers >= r1 + r2:
        return 0.0

    # Si l'un des cercles est entièrement contenu dans l'autre
    if distance_centers + min(r1, r2) <= max(r1, r2):
        return 100.0

    # Calcul de la zone d'intersection
    intersection_area = (r1**2 * math.acos((distance_centers**2 + r1**2 - r2**2) / (2 * distance_centers * r1)) +
                          r2**2 * math.acos((distance_centers**2 + r2**2 - r1**2) / (2 * distance_centers * r2)) -
                          0.5 * math.sqrt((-distance_centers + r1 + r2) * (distance_centers + r1 - r2) * (distance_centers - r1 + r2) * (distance_centers + r1 + r2)))

    # Calcul de la zone totale des deux cercles
    total_area = math.pi * r2**2

    # Calcul du pourcentage d'overlap
    overlap_percent = (intersection_area / total_area) * 100

    return overlap_percent

# Définir une fonction pour traiter les cercles qui se chevauchent
def process_overlapping_circles(image, circles):
    i=0
    # Parcourir tous les cercles
    while i < len(circles):
        # Parcourir tous les autres cercles
        j = 0
        while j < len(circles) and i < len(circles):
            if i ==j:
              j+=1
              continue
            # Calculer le pourcentage de chevauchement entre les cercles
            percentage = overlap_percentage(circles[i], circles[j])
            # Si le pourcentage de chevauchement est supérieur à 60%
            if percentage >= 60:
                # Supprimer le cercle j
                del circles[j]
                # Décrémenter j pour ajuster l'indice après la suppression
                j -= 1
            # Si le pourcentage de chevauchement est supérieur à 35%
            elif percentage >= 35:
                # Étendre le cercle i pour couvrir le cercle j
                circles[i] = ((circles[i][0][0], circles[i][0][1]), circles[i][1] + circles[j][1])

                # Supprimer le cercle j
                del circles[j]
                # Décrémenter j pour ajuster l'indice après la suppression
                j -= 1
            j+=1
        i+=1

In [123]:
import json

def calculate_circle_metrics(x1, y1, r1, x2, y2, r2, img_shape):
    # Créez des images vides pour les deux cercles, avec un seul canal
    circle1_img = np.zeros((img_shape[0], img_shape[1], 1), dtype=np.uint8)
    circle2_img = np.zeros((img_shape[0], img_shape[1], 1), dtype=np.uint8)
    
    # Dessinez les cercles sur les images
    cv2.circle(circle1_img, (int(x1), int(y1)), int(r1), 255, -1)
    cv2.circle(circle2_img, (int(x2), int(y2)), int(r2), 255, -1)
    
    # Trouvez les pixels en commun
    intersection = cv2.bitwise_and(circle1_img, circle2_img)

    # Compter les pixels dans chaque catégorie
    circle1_count = cv2.countNonZero(circle1_img)
    circle2_count = cv2.countNonZero(circle2_img)
    tp = cv2.countNonZero(intersection)
    fp = circle1_count - tp
    fn = circle2_count - tp
    
    return tp, fp, fn

def get_json_circles(json_path,image_resized,image):
    # Chargez les cercles à partir du fichier JSON
    with open(json_path, 'r') as json_file:
        data = json.load(json_file)

    # Extraire les cercles du fichier JSON
    json_circles = []
    for shape in data["shapes"]:
        if shape["shape_type"] == "circle":
            x1, y1 = shape["points"][0]
            x2, y2 = shape["points"][1]
            # Ajuster les coordonnées x, y du JSON en fonction de la nouvelle taille de l'image
            x1 = x1 * image_resized.shape[1] / image.shape[1]
            y1 = y1 * image_resized.shape[0] / image.shape[0]
            x2 = x2 * image_resized.shape[1] / image.shape[1]
            y2 = y2 * image_resized.shape[0] / image.shape[0]
            
            radius = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

            json_circles.append((x1, y1, radius))
    return json_circles

def get_value_with_label(label):
    dico = {"AUTRE":0,"1c":1,"2c":2,"5c":5,"10c":10,"20c":20,"50c":50,"1e":100,"2e":200}
    return dico[label]

def get_json_value(json_path):
    # Chargez les cercles à partir du fichier JSON
    with open(json_path, 'r') as json_file:
        data = json.load(json_file)

    # Extraire les cercles du fichier JSON
    true_value = 0
    for shape in data["shapes"]:
        true_value += get_value_with_label(shape["label"]) 
            
    return true_value

# Traitement 1: Trouver les pièces

In [124]:
def traitement(resized_image):
  #show(resized_image)
  image = resized_image
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #show(gray)
  blurred = cv2.medianBlur(gray,3)
  blurred = cv2.GaussianBlur(blurred,(5,5),4)
  #show(blurred)
  binary_image = cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,51,2)
  # show(binary_image)

  kernel_e = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
  kernel_f = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))

  binary_image_eroded = cv2.erode(binary_image, kernel_e, iterations=1)

  #image_close = cv2.morphologyEx(binary_image_eroded, cv2.MORPH_CLOSE, kernel_f,iterations=3)
  # Afficher l'image traitée
  #show(binary_image_eroded)

  contours, hierarchy = cv2.findContours(binary_image_eroded,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

  # Calculer l'aire minimale et maximale en fonction de la taille de l'image
  min_area = 100
  max_area = 10000
  circles = []
  # Dessiner les contours en filtrant par aire
  for i in range(len(contours)):
      contour_area = cv2.contourArea(contours[i])
      if contour_area > min_area and contour_area < max_area:  # Filtrer par aire min et max
          (x,y),radius = cv2.minEnclosingCircle(contours[i])
          center = (int(x),int(y))
          radius = int(radius)
          circles.append((center, radius))

  process_overlapping_circles(image, circles)
  for circle in circles:
    cv2.circle(image,circle[0],circle[1],(0,0,255),3)

  # Afficher l'image avec les contours filtrés
  # show(image)

  return circles

In [125]:
def traitement2(resized_image):
    #show(resized_image)
    image = resized_image
    
    #Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #Appliquer un flou median
    blurred_image = cv2.medianBlur(gray, 3)

    #Appliquer la binarisation adaptative
    binary_image = cv2.adaptiveThreshold(blurred_image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,51,2)

    #Elements structurants
    kernel_e = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    kernel_d = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1,2))
    kernel_f = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,4))

    #Erode et dilatation
    eroded_image = cv2.erode(binary_image, kernel_e, iterations=1)
    #dilated_image = cv2.dilate(binary_image, kernel_d, iterations=1)
    #image_close = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel_f)

    contours, _ = cv2.findContours(eroded_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #Créer une liste pour stocker les contours filtrés par taille
    filtered_contours = []

    #La taille minimale des cc à conserver
    min_contour_size = 100

    #Filtrer les contours par taille
    for contour in contours:
        if cv2.contourArea(contour) > min_contour_size:
            filtered_contours.append(contour)

    #Créer une image vide pour dessiner les contours filtrés
    #filtered_contour_image = image.copy()
    filtered_contour_image = np.ones_like(image) * 255 #une copie blanche

    #Supprimer le + grand contours (la fenetre)
    largest_contour_index = max(range(len(filtered_contours)), key=lambda i: cv2.contourArea(filtered_contours[i]))
    del filtered_contours[largest_contour_index]

    #Dessiner les contours  sur l'image vide
    cv2.drawContours(filtered_contour_image, filtered_contours, -1, (0, 255, 0), -1) #en vert
    #cv2.drawContours(filtered_contour_image, filtered_contours, -1, (0, 0, 0), thickness=2) # en noir

    #Remplir l'intérieur des pièces
    #for contour in filtered_contours:
    #    cv2.fillPoly(filtered_contour_image, pts=[contour], color=(0, 0, 0))


    #Créer une image vide pour dessiner les contours externes
    external_contour_image = image.copy() #une copie blanche

    #Trouver les contours externes des objets
    external_contours = []
    for contour in filtered_contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        external_contours.append(approx)

    #Dessiner les contours externes sur l'image vide
    cv2.drawContours(external_contour_image, external_contours, -1, (0, 255, 0), thickness=2) # en vert


    #Redimensionner l'image pour l'affichage
    #binary_image = cv2.resize(binary_image, (0,0), fx=0.3, fy=0.3)
    #image_close = cv2.resize(image_close, (0,0), fx=0.3, fy=0.3)
    #filtered_contour_image = cv2.resize(filtered_contour_image, (0,0), fx=0.3, fy=0.3)
    #external_contour_image = cv2.resize(external_contour_image, (0,0), fx=0.3, fy=0.3)

    #Afficher l'image avec les contours filtrés par taille
    show(external_contour_image) 


In [126]:
def traitement3(resized_image):
    #show(resized_image)
    image = resized_image

    # Neutralize luminance
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Darken the image
    hsv[:,:,2] = hsv[:,:,2] * 0.3  # Multiply the V channel by 0.8 to darken the image

    # Enhance the edges
    blurred = cv2.GaussianBlur(hsv[:,:,2], (55, 55), 0)
    edges = cv2.Canny(blurred, 100, 200)

    # Combine the original V channel with the enhanced edges
    hsv[:,:,2] = np.maximum(hsv[:,:,2], edges)

    # Convert back to BGR
    image_darkened = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    # Binarize the enhanced image using Otsu's thresholding
    gray_image = cv2.cvtColor(image_darkened, cv2.COLOR_BGR2GRAY)
    #binary_image = cv2.adaptiveThreshold(gray_image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,51,2)
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    show(image_darkened) 
    show(binary_image)

# Traitement 2: trouver la valeur des pièces

In [127]:
def choose_coin_value(x,y,r,image_resized):
    return 0

def estimate_coins(circles,image_resized,json_path):
    val = 0
    for circle in circles:
        val += choose_coin_value(circle[0][0],circle[0][1],circle[1],image_resized)

    return get_json_value(json_path),val

# Main

In [128]:
def check_coins_detection(circles,json_path,image,image_resized):

    json_circles = get_json_circles(json_path,image_resized,image)
    
    coins_found_json = []
    for circle in circles:
        circle_x, circle_y = circle[0]
        circle_radius = circle[1]
        best_precision = 0
        indice_coin = -1
        i = 0
        for json_circle in json_circles:

            # On a déjà trouvé cette pièce
            if i in coins_found_json:
                i += 1
                continue

            json_circle_x, json_circle_y, json_circle_radius = json_circle
            
            # Calcul des métriques de cercle
            TP,FP,FN = calculate_circle_metrics(circle_x,circle_y,circle_radius,json_circle_x, json_circle_y, json_circle_radius,image_resized.shape)

            # Calcul de la précision
            precision = TP / (TP + FP) if (TP + FP) > 0 else 0
            if precision > best_precision:
                best_precision = precision
                indice_coin = i

            #jaccard_index = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0
            
            i += 1

        # Si on a trouvé une pièce
        if(best_precision > 0.4):
            # On l'ajoute dans la liste des pièces trouvées
            coins_found_json.append(indice_coin)
    
    # Retourne le nombre de pièces, le nombre de pièces trouvées correctes et le nombre de pièces trouvées
    return len(json_circles), len(coins_found_json), len(circles)


In [129]:
folder_path = "Dataset"  # Replace with the path to your folder

import os

# Boucle sur les fichiers dans le dossier
for filename in os.listdir(folder_path):
    # Vérifiez si le fichier est une image
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".heic")):
        # Chemin complet de l'image
        image_path = os.path.join(folder_path, filename)
        
        # Chargez l'image et redimensionnez-la
        image = cv2.imread(image_path)
        image_resized = resize_image(image,500)

        # Traitement de l'image pour détecter les cercles
        circles = traitement(image_resized)
        
        # Chemin complet du fichier JSON associé
        json_filename = os.path.splitext(filename)[0] + ".json"
        json_path = os.path.join(folder_path, json_filename)
        
        # Vérification de la détection des pièces de monnaie
        true_count, correct_found, coins_found = check_coins_detection(circles, json_path, image=image,image_resized=image_resized)
        
        # Affichage des résultats pour l'image actuelle
        print("-------------------------")
        print("Image:", filename)
        print("Nombre total de pièces dans le fichier JSON:", true_count)
        print("Nombre de pièces correctement trouvées:", correct_found)
        print("Nombre de pièces trouvées mais incorrectes:", coins_found - correct_found)
        print("Nombre de pièces non trouvées:", true_count - correct_found)

        # Partie pour trouver la valeur des pieces
        true_value,value_found = estimate_coins(circles,image_resized,json_path)

        print("-------------------------")
        print("Valeur réelle des pièces dans le fichier JSON:", true_value)
        print("Valeur estimée des pièces trouvées:", value_found)

-------------------------
Image: 006.jpeg
Nombre total de pièces dans le fichier JSON: 23
Nombre de pièces correctement trouvées: 8
Nombre de pièces trouvées mais incorrectes: 6
Nombre de pièces non trouvées: 15
-------------------------
Valeur réelle des pièces dans le fichier JSON: 332
Valeur estimée des pièces trouvées: 0
-------------------------
Image: 007.jpeg
Nombre total de pièces dans le fichier JSON: 23
Nombre de pièces correctement trouvées: 9
Nombre de pièces trouvées mais incorrectes: 6
Nombre de pièces non trouvées: 14
-------------------------
Valeur réelle des pièces dans le fichier JSON: 332
Valeur estimée des pièces trouvées: 0
-------------------------
Image: 008.jpeg
Nombre total de pièces dans le fichier JSON: 14
Nombre de pièces correctement trouvées: 5
Nombre de pièces trouvées mais incorrectes: 58
Nombre de pièces non trouvées: 9
-------------------------
Valeur réelle des pièces dans le fichier JSON: 166
Valeur estimée des pièces trouvées: 0
-------------------